In [1]:
import requests
import json
from typing import Tuple, List
import pandas as pd
import math
from tqdm import tqdm

In [2]:
outpath = '../data/mktcap.csv'

UNIT = 'USD'

base_url = 'https://min-api.cryptocompare.com/data'
headers = {
    'Accepts': 'application/json',
    'Authorization': 'Apikey 2572b1c288c45f00c57f9a0dfa1a0da93afc454365cfaaf7ee17820f898fa7b7',
}

def get_price_meta_data(symbols: List[str]) -> Tuple[dict, bool]:
    success = True
    symbols_str = ','.join(symbols)
    url = base_url + f'/pricemultifull?fsyms={symbols_str}&tsyms={UNIT}'
    try:
        res = requests.get(url, headers=headers)
    except requests.exceptions.RequestException as e:
        raise SystemExit(e)
    data = json.loads(res.text)
    # check success
    if not 'RAW' in data:
        success = False
        return None, success
    else:
        return data['RAW'], success

In [3]:
# load coin IDs
with open('../data/coin_info.json') as f:
    coin_info = json.load(f)

coins = list(coin_info.keys())
len(coins)

7584

In [4]:
BATCH_SIZE_LIMIT = 1000
num_coins = len(coins)

num_batches = 0
batches = [[]]

cur_batch_size = 0
for symbol in coins:
    # add +1 for comma
    if cur_batch_size + len(symbol) + 1 <= BATCH_SIZE_LIMIT:
        cur_batch_size += len(symbol) + 1
        batches[num_batches].append(symbol)
    else:
        num_batches += 1
        cur_batch_size = len(symbol)
        batches.append([symbol])

len(batches)

39

In [5]:
row_list = []

for batch in tqdm(batches):
    symbols = batch
    data, success = get_price_meta_data(symbols)
    if success:
        for symbol, symbol_data in data.items():
            if 'USD' in symbol_data:
                price_data = symbol_data['USD']
                relevant_price_data = {
                    'symbol': symbol,
                    'supply': price_data['SUPPLY'],
                    'mktcap': price_data['MKTCAP'],
                    'mktcapPenalty': price_data['MKTCAPPENALTY'],
                    'circulatingSupply': price_data['CIRCULATINGSUPPLY'],
                    'circulatingSupplyMktcap': price_data['CIRCULATINGSUPPLYMKTCAP'],
                    'conversionType': price_data['CONVERSIONTYPE'],
                    'conversionSymbol': price_data['CONVERSIONSYMBOL']
                }
                row_list.append(relevant_price_data)
            else:
                print(f'{symbol} has no field named \'USD\'')
    else:
        print(f'Query from {symbols[0]} to {symbols[-1]} unsuccessful')

df = pd.DataFrame(row_list)
df.to_csv(outpath)

100%|██████████| 39/39 [00:11<00:00,  3.29it/s]


In [6]:
df

,symbol,supply,mktcap,mktcapPenalty,circulatingSupply,circulatingSupplyMktcap,conversionType,conversionSymbol
0,42,4.199995e+01,3.317996e+07,0.0,41.999952,3.317996e+07,direct,
1,300,3.000000e+02,6.073940e+05,0.0,0.000000,0.000000e+00,multiply,BTC
2,365,0.000000e+00,0.000000e+00,0.0,0.000000,0.000000e+00,multiply,BTC
3,404,0.000000e+00,0.000000e+00,0.0,0.000000,0.000000e+00,direct,
4,611,0.000000e+00,0.000000e+00,0.0,0.000000,0.000000e+00,multiply,BTC
...,...,...,...,...,...,...,...,...
5938,REALPLATFORM,0.000000e+00,0.000000e+00,0.0,0.000000,0.000000e+00,direct,
5939,REAL,1.000000e+08,5.235000e+08,0.0,0.000000,0.000000e+00,direct,
5940,REVU,0.000000e+00,0.000000e+00,0.0,0.000000,0.000000e+00,multiply,USDT
5941,STARL,1.000000e+13,1.691164e+08,0.0,0.000000,0.000000e+00,multiply,WETH
